In [1]:
import pandas as pd

# 2025 GP rounds (venue/country labels as commonly used in F1/fastf1 contexts)
gp_2025 = [
    "Australia", "China", "Japan", "Bahrain", "Saudi Arabia", "Miami",
    "Emilia Romagna", "Monaco", "Spain", "Canada", "Austria", "Great Britain",
    "Belgium", "Hungary", "Netherlands", "Italy", "Azerbaijan", "Singapore",
    "United States", "Mexico", "Brazil", "Las Vegas", "Qatar", "Abu Dhabi"
]

# 2025 Sprint venues (6)
sprint_2025 = ["China", "Miami", "Belgium", "United States", "Brazil", "Qatar"]

races_2025 = pd.DataFrame(
    [(2025, gp, "Race") for gp in gp_2025] +
    [(2025, gp, "Sprint") for gp in sprint_2025],
    columns=["year", "country", "session"]
).sort_values(["year", "country", "session"]).reset_index(drop=True)

races_2025

races_2025["is_sprint_weekend"] = races_2025["country"].isin(sprint_2025)
races_2025["event_key"] = races_2025["year"].astype(str) + "-" + races_2025["country"]

In [3]:
import fastf1
from fastf1 import Cache

# --- Cache  ---
Cache.enable_cache("./fastf1_cache")  

# --- Robust downloader/merger ---
def process_session(year: int, country: str, session_type: str) -> pd.DataFrame:
    """
    Downloads FastF1 laps + weather for a given (year, country/event_name, session_type),
    merges weather to laps via nearest previous weather sample (backward asof),
    and returns a tidy dataframe with metadata columns matching races dataframe.
    """
    try:
        session = fastf1.get_session(year, country, session_type)
        session.load(laps=True, weather=True, telemetry=False, messages=False)

        laps = session.laps.copy()
        weather = session.weather_data.copy()

        # Guardrails: empty sessions
        if laps is None or laps.empty:
            print(f"[SKIP] No laps: {year} {country} {session_type}")
            return pd.DataFrame()
        if weather is None or weather.empty:
            print(f"[WARN] No weather: {year} {country} {session_type}")
            # still return laps, with NA weather columns
            out = laps.copy()
            out["year"] = year
            out["country"] = country
            out["session"] = session_type
            return out

        # Normalize Time columns to timedelta
        laps["Time"] = pd.to_timedelta(laps["Time"])
        weather["Time"] = pd.to_timedelta(weather["Time"])

        # Sort for merge_asof
        laps = laps.sort_values("Time").reset_index(drop=True)
        weather = weather.sort_values("Time").reset_index(drop=True)

        # Merge weather to laps:
        merged = pd.merge_asof(
            laps,
            weather,
            on="Time",
            direction="backward",
            allow_exact_matches=True
        )

        
        merged["year"] = year
        merged["country"] = country
        merged["session"] = session_type

       

        return merged

    except Exception as e:
        print(f"[FAIL] {year} {country} {session_type}: {type(e).__name__}: {e}")
        return pd.DataFrame()


def download_2025_from_calendar(races_2025: pd.DataFrame) -> pd.DataFrame:
    """
    Expects races_2025 columns: ['year','country','session'] (initial dataframe).
    Iterates, downloads, merges, concatenates.
    """
    required = {"year", "country", "session"}
    missing = required - set(races_2025.columns)
    if missing:
        raise ValueError(f"races_2025 missing columns: {missing}")

    races_2025 = races_2025.sort_values(["year", "country", "session"]).reset_index(drop=True)

    out = []
    for i, row in races_2025.iterrows():
        y, c, s = int(row["year"]), str(row["country"]), str(row["session"])
        print(f"[{i+1}/{len(races_2025)}] Downloading: {y} {c} {s}")
        df = process_session(y, c, s)
        if not df.empty:
            out.append(df)

    if not out:
        return pd.DataFrame()

    final_df = pd.concat(out, ignore_index=True)

    meta = ["year", "country", "session"]
    cols = meta + [c for c in final_df.columns if c not in meta]
    final_df = final_df[cols]

    return final_df

# Downloading data
final_df = download_2025_from_calendar(races_2025)

print(final_df.head())
final_df.to_excel("Dataset_2025.xlsx", index=False)

[1/30] Downloading: 2025 Abu Dhabi Race


core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No 

[2/30] Downloading: 2025 Australia Race


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

[3/30] Downloading: 2025 Austria Race


...

[29/30] Downloading: 2025 United States Race


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

[30/30] Downloading: 2025 United States Sprint


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

   year    country session                   Time Driver DriverNumber  \
0  2025  Abu Dhabi    Race 0 days 00:59:52.411000    VER            1   
1  2025  Abu Dhabi    Race 0 days 00:59:53.640000    PIA           81   
2  2025  Abu Dhabi    Race 0 days 00:59:54.686000    NOR            4   
3  2025  Abu Dhabi    Race 0 days 00:59:55.358000    LEC           16   
4  2025  Abu Dhabi    Race 0 days 00:59:55.714000    ALO           14   

                 LapTime  LapNumber  Stint PitOutTime  ... DeletedReason  \
0 0 days 00:01:31.677000        1.0    1.0        NaT  ...                 
1 0 days 00:01:32.906000        1.0    1.0        NaT  ...                 
2 0 days 00:01:33.952000        1.0    1.0        NaT  ...                 
3 0 days 00:01:34.624000        1.0    1.0        NaT  ...                 
4 0 days 00:01:34.980000        1.0    1.0        NaT  ...                 

  FastF1Generated IsAccurate AirTemp Humidity Pressure Rainfall  TrackTemp  \
0           False      Fal